In [1]:
import glob
import os
import pickle
import numpy

import time

import pandas as pd

In [2]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 500)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
DATA_PATH="data/"

In [5]:
glob.glob(DATA_PATH + "*.xlsx")

['data/teste_normas_new_full_20250122_114126.xlsx',
 'data/ragas_total_normas_full_alternative_references_20250122_010614.xlsx',
 'data/teste_normas_full_20250122_010614.xlsx',
 'data/ragas_total_normas_new_full_alternative_references_20250122_114126.xlsx',
 'data/missing_references_urls_20250216_final.xlsx',
 'data/teste_normas_acordaos_full_20250122_010614.xlsx',
 'data/ragas_total_normas_acórdãos_full_alternative_references_20250122_010614.xlsx']

In [7]:
new_segmentation_df = pd.read_excel(DATA_PATH + 'teste_normas_new_full_20250122_114126.xlsx')

In [8]:
new_segmentation_df.columns

Index(['question_number', 'question_text', 'answer_cleaned', 'resposta_rag',
       'contexto'],
      dtype='object')

In [10]:
new_segmentation_df.iloc[0]['resposta_rag']

'A obrigatoriedade de apresentação da Declaração de Ajuste Anual relativa ao exercício de 2024, ano-calendário de 2023, recai sobre as pessoas físicas que se enquadram em uma das seguintes situações:\n\n1. Receberam rendimentos tributáveis superiores ao limite estabelecido pela Receita Federal.\n2. Obtiveram rendimentos isentos, não tributáveis ou tributados na fonte cuja soma seja superior ao limite determinado.\n3. Tiveram ganho de capital na venda de bens ou operações de bolsa.\n4. Realizaram operações em bolsas de valores.\n5. Detinham bens ou direitos com valor total superior a determinado patamar.\n\nEssas diretrizes determinam quem precisa apresentar a declaração, visando à regularização fiscal e à prestação de contas junto à Receita Federal.\n\nLei nº 13.254, art. 4º; Instrução Normativa RFB nº 1.704, art. 19; Decreto Nº 9.580, art. 557.'

In [102]:
new_segmentation_df['contexto']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [11]:
import ast
import re

In [96]:
spliting_hierarchy=[
    "LIVRO ([IVX]+)",
    "TÍTULO ([IVX]+)",
    "(CAPÍTULO)|(Capítulo)\s([IVX]+|ÚNICO)",
    # "CAPÍTULO ([IVX]+)|Capítulo ([IVX]+)|CAPÍTULO (ÚNICO)",
    "Seção ([IVX]+)",
    "Subseção (.+)",
    # "\nArt\. \d+[º\. ]*|\n.\s+Art\. \d+[º\. ]*|\nArtigo único\.",
    "Art\. (\d+)[º\. ]*",
    "§ (\d+)[º\. ]*",
    "([IVX])+[-\s]*",
]

hierarchy_names=[
    "livro",
    "título",
    "capítulo",
    "seção",
    "subseção",
    "artigo",
    "parágrafo",
    "inciso"
]

In [103]:
def extract_question_references(rag_output_df):
    questions = rag_output_df['question_number'].to_numpy()
    contexts = rag_output_df['contexto'].to_numpy()

    questions_references = []

    for i in range(len(contexts)):
        passages_list = ast.literal_eval(contexts[i])

        for passage in passages_list:

            passage_info = {}
            passage_info['question'] = questions[i]        
            
            passage_info['path'] = re.split(": \n|: ", passage)[0]

            passage_path_parts = re.split("_", passage_info['path'])

            print(f"passage_path_parts={passage_path_parts}")

            passage_info['nome'] = passage_path_parts[0]
            
            current_level = 0
            for path_details in passage_path_parts[1:]:
                for j, hierarchy_pattern in enumerate(spliting_hierarchy[current_level:]):

                    print(f"current_level={current_level}, j={j}")
                    
                    passage_info[hierarchy_names[current_level + j]] = ""
                    
                    if re.match(hierarchy_pattern, path_details) is not None:
                        parts = re.split(hierarchy_pattern, path_details)

                        print(f"{hierarchy_names[current_level + j]} = {parts}")
                        
                        if hierarchy_names[current_level + j] == 'capítulo':
                            passage_info[hierarchy_names[current_level + j]] = parts[-1]
                        else:
                            passage_info[hierarchy_names[current_level + j]] = parts[1]

                        current_level += j + 1

                        break

            for current_level in range(current_level,len(hierarchy_names)):
                passage_info[hierarchy_names[current_level]] = ""                        
                
            questions_references.append(passage_info)

    return questions_references

In [104]:
new_segmentation_processed = extract_question_references(new_segmentation_df)

passage_path_parts=['Lei nº 13.254', 'Art. 4º', '§ 2º']
current_level=0, j=0
current_level=0, j=1
current_level=0, j=2
current_level=0, j=3
current_level=0, j=4
current_level=0, j=5
artigo = ['', '4', '']
current_level=6, j=0
parágrafo = ['', '2', '']
passage_path_parts=['Instrução Normativa RFB nº 1.704', 'CAPÍTULO II', 'Seção IV', 'Art. 19.']
current_level=0, j=0
current_level=0, j=1
current_level=0, j=2
capítulo = ['', 'CAPÍTULO', None, None, ' II']
current_level=3, j=0
seção = ['', 'IV', '']
current_level=4, j=0
current_level=4, j=1
artigo = ['', '19', '']
passage_path_parts=['RIR2018', 'LIVRO II', 'TÍTULO VIII', 'CAPÍTULO XI', 'Art. 557.', 'preamble']
current_level=0, j=0
livro = ['', 'II', '']
current_level=1, j=0
título = ['', 'VIII', '']
current_level=2, j=0
capítulo = ['', 'CAPÍTULO', None, None, ' XI']
current_level=3, j=0
current_level=3, j=1
current_level=3, j=2
artigo = ['', '557', '']
current_level=6, j=0
current_level=6, j=1
passage_path_parts=['Lei nº 13.254', 'Art. 4º'

In [105]:
len(new_segmentation_processed)

1731

In [106]:
pd.DataFrame(new_segmentation_processed)

,question,path,nome,livro,título,capítulo,seção,subseção,artigo,parágrafo,inciso
0,1,Lei nº 13.254_Art. 4º_§ 2º,Lei nº 13.254,,,,,,4,2,
1,1,Instrução Normativa RFB nº 1.704_CAPÍTULO II_Seção IV_Art. 19.,Instrução Normativa RFB nº 1.704,,,II,IV,,19,,
2,1,RIR2018_LIVRO II_TÍTULO VIII_CAPÍTULO XI_Art. 557._preamble,RIR2018,II,VIII,XI,,,557,,
3,2,Lei nº 13.254_Art. 4º_§ 2º,Lei nº 13.254,,,,,,4,2,
4,2,Instrução Normativa RFB nº 1.704_CAPÍTULO II_Seção IV_Art. 19.,Instrução Normativa RFB nº 1.704,,,II,IV,,19,,
...,...,...,...,...,...,...,...,...,...,...,...
1726,714,Lei nº 8.383,Lei nº 8.383,,,,,,,,
1727,714,Instrução Normativa RFB nº 1.585,Instrução Normativa RFB nº 1.585,,,,,,,,
1728,715,Instrução Normativa RFB nº 1.585,Instrução Normativa RFB nº 1.585,,,,,,,,
1729,715,Instrução Normativa RFB nº 1.585,Instrução Normativa RFB nº 1.585,,,,,,,,


In [108]:
with open(DATA_PATH + "passage_paths_teste_normas_new_full_20250122_114126.pkl", "wb") as output_file:
    pickle.dump(new_segmentation_processed, output_file, pickle.HIGHEST_PROTOCOL)

In [17]:
t1 = re.split("_", "Instrução Normativa RFB nº 1.704_CAPÍTULO II_Seção IV_Art. 19.")

In [18]:
t1

['Instrução Normativa RFB nº 1.704', 'CAPÍTULO II', 'Seção IV', 'Art. 19.']

In [19]:
t1[1]

'CAPÍTULO II'

In [44]:
re.split(spliting_hierarchy[2], t1[1])

['', 'CAPÍTULO', None, None, ' II']

In [47]:
re.split(spliting_hierarchy[3], t1[2])

['', 'IV', '']

In [38]:
re.split(spliting_hierarchy[2], "CAPÍTULO ÚNICO")

['', 'CAPÍTULO', None, None, ' ÚNICO']